In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from keras_vggface.vggface import VGGFace
from classification_models.tfkeras import Classifiers

In [3]:
ResNeXt50, preprocess_input = Classifiers.get("resnext50")
resnext = ResNeXt50(include_top=False, input_shape=(224, 224, 3), weights=None, pooling="avg")

layer1 = resnext.get_layer("stage3_unit6_relu")
layer2 = resnext.get_layer("stage4_unit1_conv1")

base = tf.keras.models.Model(resnext.input, layer1.output)
conv = tf.keras.models.Model(layer2.input, resnext.output)

conv.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14, 14, 102  0           []                               
                                4)]                                                               
                                                                                                  
 stage4_unit1_conv1 (Conv2D)    (None, 14, 14, 1024  1048576     ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stage4_unit1_bn1 (BatchNormali  (None, 14, 14, 1024  4096       ['stage4_unit1_conv1[1][0]']     
 zation)                        )                                                           

In [3]:
resnet = VGGFace(
        model="resnet50",
        include_top=False,
        input_shape=(224, 224, 3),
        weights="vggface"
    )

resnet.summary()

Model: "vggface_resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                  

In [4]:
vgg = VGGFace(
        model="vgg16",
        include_top=False,
        input_shape=(224, 224, 3),
        pooling="None",
        weights="vggface"
    )

vgg.summary()

Model: "vggface_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 pool1 (MaxPooling2D)        (None, 112, 112, 64)      0         
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                                 
 pool2 (MaxPooling2D)        (None, 56, 56, 128)     

In [31]:
resnet = VGGFace(
    model="resnet50",
    include_top=False,
    input_shape=(224, 224, 3),
    weights="vggface"
)

#vgg = VGGFace(
    #model="vgg16",
    #include_top=False,
    #input_shape=(224, 224, 3),
    #pooling="None",
    #weights="vggface"
#)

#vgg = tf.keras.models.Model(vgg.get_layer("conv5_1").input, vgg.output)

inputs = layers.Input(shape=(14,14,1024))
x = tf.keras.layers.Conv2D(512, 3, strides=1, padding="same")(inputs)
x = tf.keras.layers.Conv2D(512, 3, strides=1, padding="same")(x)
x = tf.keras.layers.Conv2D(512, 3, strides=1, padding="same")(x)
x = tf.keras.layers.MaxPooling2D(2, strides=2)(x)
x = layers.AveragePooling2D((7, 7))(x)
trainable = tf.keras.models.Model(inputs, x)

base = tf.keras.models.Model(resnet.input, outputs=resnet.layers[-34].output)

Model: "model_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_92 (InputLayer)       [(None, 14, 14, 1024)]    0         
                                                                 
 conv2d_179 (Conv2D)         (None, 14, 14, 512)       4719104   
                                                                 
 conv2d_180 (Conv2D)         (None, 14, 14, 512)       2359808   
                                                                 
 conv2d_181 (Conv2D)         (None, 14, 14, 512)       2359808   
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 7, 7, 512)        0         
 g2D)                                                            
                                                                 
 average_pooling2d_14 (Avera  (None, 1, 1, 512)        0         
 gePooling2D)                                             

In [ ]:
a = tf.keras.losses.MeanSquaredError()
b = losses.SquaredEarthMoversDistance()

alpha = K.variable(1.)

def wildloss(y_true, y_pred):
    return alpha*a(y_true, y_pred) + (1-alpha)*b(y_true, y_pred)

class UpdateLoss(tf.keras.callbacks.Callback):
    def __init__(self, alpha):
        self.alpha = alpha
        self.max_pc = 0

    def on_epoch_end(self, epoch, logs=None):
        print(self.alpha)
        """
        pc = logs["val_correlation"]

        if pc > self.max_pc:
            if 1-self.max_pc/pc < 0.001:
                K.set_value(self.alpha, np.max([0, self.alpha-0.1]))

            else:
                K.set_value(self.alpha, np.min([1, self.alpha+0.1]))

            self.max_pc = pc

        print(self.alpha)
        """

        #K.set_value(self.alpha, np.max([0, 0.7-0.4*(epoch/100)**2]))
        #Sine law
        #K.set_value(self.alpha, max*np.sin(epoch*np.pi/40)**2)
        #Parabolic law
        K.set_value(self.alpha, np.max([0, 1-((epoch/35)**2)]))

#loss = wildloss